### Import libraries and environment variables

In [2]:
from typing import List
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.core.ingestion import IngestionPipeline
from llama_index.core.schema import BaseNode, TransformComponent
from llama_index.vector_stores.faiss import FaissVectorStore
from llama_index.core.text_splitter import SentenceSplitter
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings
import faiss
import os
import sys
from dotenv import load_dotenv
import hashlib
import pickle
import json
from deepeval import evaluate
from deepeval.metrics import GEval, FaithfulnessMetric, ContextualRelevancyMetric
from deepeval.test_case import LLMTestCaseParams, LLMTestCase
from llama_index.core.indices.query.query_transform.base import (
    HyDEQueryTransform,
)
from llama_index.core.query_engine import TransformQueryEngine


sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..'))) # Add the parent directory to the path sicnce we work with notebooks

EMBED_DIMENSION = 512

# Chunk settings are way different than langchain examples
# Beacuse for the chunk length langchain uses length of the string,
# while llamaindex uses length of the tokens
CHUNK_SIZE = 200
CHUNK_OVERLAP = 50

# Load environment variables from a .env file
load_dotenv()

# Set the OpenAI API key environment variable
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

# Set embeddig model on LlamaIndex global settings
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small", dimensions=EMBED_DIMENSION)

/Users/user/Projects/ragbrag_pycon_ie_24/venv/lib/python3.10/site-packages/deepeval/__init__.py:49: UserWarning: You are using deepeval version 1.2.9, however version 1.3.5 is available. You should consider upgrading via the "pip install --upgrade deepeval" command.
  warnings.warn(
/Users/user/Projects/ragbrag_pycon_ie_24/venv/lib/python3.10/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in HuggingFaceInferenceAPIEmbeddings has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


### Read Docs

In [3]:
path = "../data/"
node_parser = SimpleDirectoryReader(input_dir=path, required_exts=['.txt', '.pdf'])
documents = node_parser.load_data()
print(documents[0])

Doc ID: bd82f4c1-05ad-405e-998b-70ecc87b6f44
Text: Promoted by SNP 3 Jacksons Entry EH8 8PJ. Printed by Saltire 60
Brook Street G40 2AB.“A FUTURE    MADE IN    SCOTLAND.” VOTE SNP  FOR
SCOTLAND


### Text Cleaner Transformation

In [4]:
class TextCleaner(TransformComponent):
    """
    Transformation to be used within the ingestion pipeline.
    Cleans clutters from texts.
    """
    def __call__(self, nodes, **kwargs) -> List[BaseNode]:
        
        for node in nodes:
            node.text = node.text.replace('\t', ' ') # Replace tabs with spaces
            node.text = node.text.replace(' \n', ' ') # Replace paragraph seperator with spacaes
            
        return nodes

In [5]:
def hash_documents(documents):
    # combine all the texts into a single string
    all_titles = [doc.metadata['file_name'] for doc in documents]
    all_titles_distinct = list(set(all_titles))
    all_titles_distinct.sort()
    all_titles_str = " ".join(all_titles_distinct)
    # return a hash of the combined text which will stay consistent if the text is the same across multiple runs
    return hashlib.md5(all_titles_str.encode('utf-8')).hexdigest()

### Vector Store

In [13]:
CACHE_DIR = "../cache"
VECTOR_STORE_PATH = os.path.join(CACHE_DIR, "faiss_index.pkl")
HASH_PATH = os.path.join(CACHE_DIR, "documents_hash.txt")

def load_or_create_vector_store(documents, embed_dim, chunk_size, chunk_overlap):
    os.makedirs(CACHE_DIR, exist_ok=True)
    
    current_hash = hash_documents(documents)
    
    if os.path.exists(HASH_PATH) and os.path.exists(VECTOR_STORE_PATH):
        with open(HASH_PATH, 'r') as f:
            stored_hash = f.read().strip()

        if stored_hash == current_hash:
            print("Loading vector store from cache...")
            with open(VECTOR_STORE_PATH, 'rb') as f:
                return pickle.load(f)
    
    print("Creating new vector store...")
    faiss_index = faiss.IndexFlatL2(embed_dim)
    vector_store = FaissVectorStore(faiss_index=faiss_index)
    
    text_splitter = SentenceSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    
    pipeline = IngestionPipeline(
        transformations=[
            TextCleaner(),
            text_splitter,
        ],
        vector_store=vector_store,
    )
    
    nodes = pipeline.run(documents=documents)
    vector_store_index = VectorStoreIndex(nodes)
    
    # Save the new vector store and hash
    with open(VECTOR_STORE_PATH, 'wb') as f:
        pickle.dump(vector_store_index, f)
    
    with open(HASH_PATH, 'w') as f:
        f.write(current_hash)
    
    return vector_store_index

In [62]:
vector_store_index = load_or_create_vector_store(documents, EMBED_DIMENSION, CHUNK_SIZE, CHUNK_OVERLAP)

Loading vector store from cache...


### Create retriever

In [63]:
retriever = vector_store_index.as_retriever(similarity_top_k=2)

### Test retriever

In [64]:
def show_context(context):
    """
    Display the contents of the provided context list.

    Args:
        context (list): A list of context items to be displayed.

    Prints each context item in the list with a heading indicating its position.
    """
    for i, c in enumerate(context):
        print(f"Context {i+1}:")
        print(c.text)
        print("\n")

In [65]:
test_query = "What is the SNP's policy on climate change?"
context = retriever.retrieve(test_query)
show_context(context)

Context 1:
20BUILDING A FAIRER, GREENER ECONOMY Under the SNP, Scotland’s economy is already one of the best performing parts of the UK with both GDP per head and productivity growing faster in Scotland than the UK as a whole.  But we want to go further. Our commitment to tackling the twin crises of climate change and nature loss is unwavering and we believe emissions reduction and economic prosperity go hand in hand. We want  to share in the enormous economic opportunities of the global transition to net zero. SNP MPs will demand the UK Government:
Bring forward an immediate emergency budget following the election to reverse cuts to public spending and deliver meaningful investment in economic growth, including green energy.


Context 2:
Only in this way, can as many jobs as possible be protected and the transition made to working on the new, greener technologies that offer a long-term future for the site..
Rule out new nuclear power plants in Scotland. The SNP believe the best pathwa

### Hypothetical Document Embedding (HyDE)

In [78]:

from llama_index.core.query_pipeline import QueryPipeline

class HyDERetriever:
    def __init__(self, chunk_size=250, chunk_overlap=50):
        self.llm = OpenAI(temperature=0, model_name="gpt-4o", max_tokens=4000)

        self.embeddings = Settings.embed_model
        self.chunk_size = chunk_size
        self.chunk_overlap = chunk_overlap
        self.vectore_store_retriever = retriever    
        
        self.hyde_prompt = PromptTemplate(
            """Given the question '{query}', generate a hypothetical document that directly answers this question. The document should be detailed and in-depth.
            the document size has be exactly {chunk_size} characters.""",
        )
        self.hyde_chain = QueryPipeline(chain=[self.hyde_prompt, self.llm], verbose=True)

    def generate_hypothetical_document(self, query):
        return self.hyde_chain.run(query=query, chunk_size=self.chunk_size)

    def retrieve(self, query):
        hypothetical_doc = self.generate_hypothetical_document(query)
        similar_docs = self.vectore_store_retriever.retrieve(query)
        return similar_docs, hypothetical_doc

In [79]:
hyde_retriever = HyDERetriever(path)

In [80]:
test_query = "What is the SNP's policy on climate change?"
results, hypothetical_doc = hyde_retriever.retrieve(test_query)

> Running module f2b4cbd9-8b2c-418d-b3ad-7941d5bc2c62 with input: 
query: What is the SNP's policy on climate change?
chunk_size: ../data/

> Running module 89a5230b-fca2-4cf6-bd73-287d92c96ebb with input: 
messages: Given the question 'What is the SNP's policy on climate change?', generate a hypothetical document that directly answers this question. The document should be detailed and in-depth.
            the do...



In [81]:
hypothetical_doc

ChatResponse(message=ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content="The Scottish National Party (SNP) has a strong commitment to tackling climate change and has set ambitious targets to reduce greenhouse gas emissions. The SNP's policy on climate change is centered around transitioning to a low-carbon economy, investing in renewable energy sources, and promoting sustainable practices across all sectors.\n\nOne of the key initiatives of the SNP is the Climate Change (Emissions Reduction Targets) (Scotland) Act, which sets legally binding targets to reduce emissions by at least 75% by 2030 and achieve net-zero emissions by 2045. The SNP also supports the UK's commitment to achieving net-zero emissions by 2050 and has called for increased action at the international level to address the global climate crisis.\n\nIn terms of energy policy, the SNP has prioritized the development of renewable energy sources such as wind, solar, and hydro power. The party has set ambitious t

In [82]:
results

[NodeWithScore(node=TextNode(id_='ab18b66f-c381-425a-8109-e246e75efc45', embedding=None, metadata={'page_label': '22', 'file_name': '2024-06-20b-SNP-General-Election-Manifesto-2024_interactive.pdf', 'file_path': '/Users/user/Projects/ragbrag_pycon_ie_24/notebooks/../data/2024-06-20b-SNP-General-Election-Manifesto-2024_interactive.pdf', 'file_type': 'application/pdf', 'file_size': 3559498, 'creation_date': '2024-09-24', 'last_modified_date': '2024-09-24'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='edc800ff-6ef2-47dd-8496-76e3b4566c69', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'page_label': '22', 'file_name': '2024-06-20b-SNP-General-Election-Manifesto-2024_interactive.pdf', 'file_path': '/Users/